<a href="https://colab.research.google.com/github/mapzine123/machine-running/blob/main/3_%ED%9B%88%EB%A0%A8_%EB%85%B8%ED%95%98%EC%9A%B0_%EC%9D%B5%ED%9E%88%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 훈련 노하우 배우기

"어느 데이터 세트에만 치우친 모델을 만들지 않는 것"

In [2]:
# 로지스틱 회귀로 모델 훈련하고 평가
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier

cancer = load_breast_cancer()
x = cancer.data
y = cancer.target

x_train_all, x_test, y_train_all, y_test = train_test_split(x, y, stratify=y, test_size=0.2, random_state=42)

sgd = SGDClassifier(loss='log', random_state=42)
sgd.fit(x_train_all, y_train_all)
sgd.score(x_test, y_test)




/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


0.8333333333333334

In [3]:
# 서포트 벡터 머신으로 모델 훈련하고 평가하기
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(loss='hinge', random_state=42)
sgd.fit(x_train_all, y_train_all)
sgd.score(x_test, y_test)

0.9385964912280702

## 결과
로지스틱 회귀의 성능 평가 결과는 83%  
서포트 벡터 머신의 성능 평가 결과는 93%  

성능이 만족스럽지 않을 경우 loss 매개변수에 다른 값을 적용할 수 있음  
이런 작업을 `모델을 튜닝한다`고 함

## 문제점
### 테스트 세트로 모델을 튜닝하면 실전에서 좋은 성능을 못낼 수 있음  

테스트 세트는 실전에 투입된 모델의 성능을 측정하기 위해 사용되는데, 테스트 세트로 모델을 튜닝하면 `테스트 세트에 대해서만` 좋은 성능을 보여주는 모델이 만들어짐  

이는 학생에게 문제의 답안지만 달달 외우게 한 것과 비슷함  

즉, 답안지에 있는 문제는 잘 푸는데 문제가 바뀌면 잘 못푸는 것과 같음  

이런 현상을 `테스트 세트의 정보가 모델에 새어 나갔다` 라고 표현함

## 정리
테스트 세트로 모델을 튜닝하면, 테스트 세트의 정보가 모델에 새어나가 모델의 일반화 성능이 왜곡됨

## 문제점에 대한 솔루션
### 모델을 튜닝할 때 테스트 세트를 사용하지 않으면 됨
테스트 세트는 모델 튜닝을 모두 마치고 실전에 투입하기 전에 딱 한번만 사용하는 것이 좋음  

모델 튜닝을 위한 세트는 따로 준비해야함  

모델 튜닝을 위한 세트는 검증 세트(validation set)라고 부르고, 훈련 세트를 조금 떼어 만듬

## 실습

### 1. 데이터 세트 준비하기

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

cancer = load_breast_cancer()

x = cancer.data
y = cancer.target

x_train_all, x_test, y_train_all, y_test = train_test_split(x, y, stratify=y, test_size=0.2, random_state=42)

### 2. 검증 세트 분할하기
훈련 : 검증 : 테스트 = 6 : 2 : 2

바로 6 : 2 : 2로 나누는게 아니라 전체 데이터를 8:2로 나누고, 다시 훈련세트를 8:2로 나누어 훈련세트와 검증세트를 만듬  

전체 데이터가 100이면 64:20:16으로 나눠짐